In [1]:
import sys
sys.path.insert(1, '../src')

import util
import pandas as pd
import numpy as np 
import warnings
from pandarallel import pandarallel
import dask.dataframe as dd
pandarallel.initialize()

INFO: Pandarallel will run on 14 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [2]:
warnings.filterwarnings('ignore')
pd.options.display.float_format = '{:,.2f}'.format
pd.set_option("display.max_rows", 999)
pd.set_option("display.max_columns", None)
pd.set_option("precision", 3)
%pylab inline
%config InlineBackend.figure_formats = ['retina']

Populating the interactive namespace from numpy and matplotlib


# **1. Read the data**

In [3]:
fn = '../data/US_Accidents_Dec20.csv'
df = pd.read_csv(fn)
df.head(3)

,ID,Source,TMC,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),Description,Number,Street,Side,City,County,State,Zipcode,Country,Timezone,Airport_Code,Weather_Timestamp,Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Direction,Wind_Speed(mph),Precipitation(in),Weather_Condition,Amenity,Bump,Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,A-1,MapQuest,201.00,3,2016-02-08 05:46:00,2016-02-08 11:00:00,39.87,-84.06,nan,nan,0.01,Right lane blocked due to accident on I-70 Eas...,nan,I-70 E,R,Dayton,Montgomery,OH,45424,US,US/Eastern,KFFO,2016-02-08 05:58:00,36.90,nan,91.00,29.68,10.00,Calm,nan,0.02,Light Rain,False,False,False,False,False,False,False,False,False,False,False,False,False,Night,Night,Night,Night
1,A-2,MapQuest,201.00,2,2016-02-08 06:07:59,2016-02-08 06:37:59,39.93,-82.83,nan,nan,0.01,Accident on Brice Rd at Tussing Rd. Expect del...,"2,584.00",Brice Rd,L,Reynoldsburg,Franklin,OH,43068-3402,US,US/Eastern,KCMH,2016-02-08 05:51:00,37.90,nan,100.00,29.65,10.00,Calm,nan,0.00,Light Rain,False,False,False,False,False,False,False,False,False,False,False,False,False,Night,Night,Night,Day
2,A-3,MapQuest,201.00,2,2016-02-08 06:49:27,2016-02-08 07:19:27,39.06,-84.03,nan,nan,0.01,Accident on OH-32 State Route 32 Westbound at ...,nan,State Route 32,R,Williamsburg,Clermont,OH,45176,US,US/Eastern,KI69,2016-02-08 06:56:00,36.00,33.30,100.00,29.67,10.00,SW,3.50,nan,Overcast,False,False,False,False,False,False,False,False,False,False,False,True,False,Night,Night,Day,Day


In [4]:
#change column labels to lowercase for convenience
df.columns = df.columns.str.lower()
df.head(3)

,id,source,tmc,severity,start_time,end_time,start_lat,start_lng,end_lat,end_lng,distance(mi),description,number,street,side,city,county,state,zipcode,country,timezone,airport_code,weather_timestamp,temperature(f),wind_chill(f),humidity(%),pressure(in),visibility(mi),wind_direction,wind_speed(mph),precipitation(in),weather_condition,amenity,bump,crossing,give_way,junction,no_exit,railway,roundabout,station,stop,traffic_calming,traffic_signal,turning_loop,sunrise_sunset,civil_twilight,nautical_twilight,astronomical_twilight
0,A-1,MapQuest,201.00,3,2016-02-08 05:46:00,2016-02-08 11:00:00,39.87,-84.06,nan,nan,0.01,Right lane blocked due to accident on I-70 Eas...,nan,I-70 E,R,Dayton,Montgomery,OH,45424,US,US/Eastern,KFFO,2016-02-08 05:58:00,36.90,nan,91.00,29.68,10.00,Calm,nan,0.02,Light Rain,False,False,False,False,False,False,False,False,False,False,False,False,False,Night,Night,Night,Night
1,A-2,MapQuest,201.00,2,2016-02-08 06:07:59,2016-02-08 06:37:59,39.93,-82.83,nan,nan,0.01,Accident on Brice Rd at Tussing Rd. Expect del...,"2,584.00",Brice Rd,L,Reynoldsburg,Franklin,OH,43068-3402,US,US/Eastern,KCMH,2016-02-08 05:51:00,37.90,nan,100.00,29.65,10.00,Calm,nan,0.00,Light Rain,False,False,False,False,False,False,False,False,False,False,False,False,False,Night,Night,Night,Day
2,A-3,MapQuest,201.00,2,2016-02-08 06:49:27,2016-02-08 07:19:27,39.06,-84.03,nan,nan,0.01,Accident on OH-32 State Route 32 Westbound at ...,nan,State Route 32,R,Williamsburg,Clermont,OH,45176,US,US/Eastern,KI69,2016-02-08 06:56:00,36.00,33.30,100.00,29.67,10.00,SW,3.50,nan,Overcast,False,False,False,False,False,False,False,False,False,False,False,True,False,Night,Night,Day,Day


# **2. Generate the name of day and season**

In [5]:
# separate start time to day name and discretized time
df['start_time_dayname'] = pd.to_datetime(df['start_time'].parallel_apply(util.get_date)).dt.day_name().str.lower()
df['start_time_daypart'] = util.get_day_parts(df['start_time'])
df['weather_timestamp'] = util.get_day_parts(df['start_time'])
df['season'] = util.get_seasons(df['start_time'])

# **3. Calculate traffic time delay**

In [6]:
# covnert string to pandas datetime type
df['start_time'] = pd.to_datetime(df['start_time'])
df['end_time'] = pd.to_datetime(df['end_time'])
# calculate duration in minute
df['duration'] = df.end_time - df.start_time 
df['duration'] = df['duration'].parallel_apply(lambda x:round(x.total_seconds() / 60) )

# **4. Column 'side' has blank rows, therfore, drop these rows**

In [7]:
# drop blank rows for 'side'
idx = df[df['side'] == ' '].index
df.drop(index=idx, inplace=True)

# **5. Process the rows with missing values**

In [8]:
missing = df.isnull().sum().reset_index()
missing.columns = ['Feature', 'Missing_Num']
missing['Missing_Percent(%)'] = 100*missing['Missing_Num']/df.shape[0]
missing.loc[missing['Missing_Percent(%)']>0,:].sort_values(by=['Missing_Num'], ascending=False)

,Feature,Missing_Num,Missing_Percent(%)
9,end_lng,2716476,64.23
8,end_lat,2716476,64.23
12,number,2686191,63.51
30,precipitation(in),2065401,48.83
24,wind_chill(f),1895870,44.83
2,tmc,1512917,35.77
29,wind_speed(mph),479212,11.33
27,visibility(mi),98557,2.33
31,weather_condition,98282,2.32
25,humidity(%),95346,2.25


## **5.1 Fill nans with the mean value of all available pressure values** 

In [9]:
# fillna for pressure
df['pressure(in)_na'] = False
df.loc[df['pressure(in)'].isnull(),'pressure(in)_na'] = True
df['pressure(in)'] = df['pressure(in)'].fillna(df['pressure(in)'].mean())

In [11]:
# add a feature for missing value
# fillna for the feature, for each state and each season, fill nans with average values
df = util.my_fillna(df, 'precipitation(in)')
df = util.my_fillna(df, 'visibility(mi)')
df = util.my_fillna(df, 'temperature(f)')
df = util.my_fillna(df, 'humidity(%)')
df = util.my_fillna(df, 'wind_speed(mph)')

# **6. Drop useless columns**

In [17]:
# drop unnecessary columns
# Turning_Loop all False
df= df.drop(['id', 'source', 'tmc', 'end_time', 
             'end_lat', 'end_lng',
             'number', 'county', 'country',
             'timezone', 'airport_code',
             'wind_chill(f)', 'turning_loop',
             'sunrise_sunset', 'civil_twilight',
             'nautical_twilight', 'astronomical_twilight',
             'weather_timestamp', 'city', 'zipcode',
             'description', 'start_time', 'street'
            ], axis=1)


# **7. Reduce file size by setting type of numbers as float16**

In [19]:
# reduce file size
df[['start_lat', 'start_lng', 'temperature(f)', 
    'humidity(%)', 'pressure(in)', 'visibility(mi)',
    'wind_speed(mph)', 'precipitation(in)'
   ]] = df[['start_lat', 'start_lng', 'temperature(f)', 
    'humidity(%)', 'pressure(in)', 'visibility(mi)',
    'wind_speed(mph)', 'precipitation(in)'
   ]].astype(np.float16)

# **8. Save a csv file for next section using dask**

In [20]:
df.to_csv('../data/temp.csv', index=False)

# **9. Set up dask cluster**

In [1]:
import dask.dataframe as dd
from dask.distributed import Client, LocalCluster
cluster = LocalCluster()
client = Client(cluster)
client


Client Scheduler: tcp://127.0.0.1:35123 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 7 Cores: 14 Memory: 20.99 GB


# **10. Read the saved csv file in step 8**

In [2]:
df = dd.read_csv('../data/temp.csv')

In [3]:
df = df.repartition(npartitions=36)
df.npartitions

36

In [4]:
df = df.reset_index(drop=True)

# **11. Extract representative wind direction features**
### Here, features include north, south, west, east, north east, south east, north west, south west, variable, calm, and na. 

In [4]:
# fill wind direction with string NA and add categorical value
df['wind_direction'] = df['wind_direction'].fillna('na').str.lower()
df['wind_direction2'] = 0 

# for north
df['wind_direction2'] = df['wind_direction2'].mask((df['wind_direction']=='nnw')|
                                                   (df['wind_direction']=='nne')|
                                                   (df['wind_direction']=='north')|
                                                   (df['wind_direction']=='n'),
                                                   'n')
# for south
df['wind_direction2'] = df['wind_direction2'].mask((df['wind_direction']=='ssw')|
                                                   (df['wind_direction']=='sse')|
                                                   (df['wind_direction']=='south')|
                                                   (df['wind_direction']=='s'),
                                                   's')

df['wind_direction2'] = df['wind_direction2'].mask((df['wind_direction']=='wnw')|
                                                   (df['wind_direction']=='wsw')|
                                                   (df['wind_direction']=='west')|
                                                   (df['wind_direction']=='w'),
                                                   'w')

df['wind_direction2'] = df['wind_direction2'].mask((df['wind_direction']=='ene')|
                                                   (df['wind_direction']=='ese')|
                                                   (df['wind_direction']=='east')|
                                                   (df['wind_direction']=='e'),
                                                   'e')

df['wind_direction2'] = df['wind_direction2'].mask(df['wind_direction']=='ne', 'ne')
df['wind_direction2'] = df['wind_direction2'].mask(df['wind_direction']=='se', 'se')
df['wind_direction2'] = df['wind_direction2'].mask(df['wind_direction']=='nw', 'nw')
df['wind_direction2'] = df['wind_direction2'].mask(df['wind_direction']=='sw', 'sw')
df['wind_direction2'] = df['wind_direction2'].mask(df['wind_direction']=='variable', 'var')
df['wind_direction2'] = df['wind_direction2'].mask(df['wind_direction']=='var', 'var')
df['wind_direction2'] = df['wind_direction2'].mask(df['wind_direction']=='calm', 'calm')
df['wind_direction2'] = df['wind_direction2'].mask(df['wind_direction']=='na', 'na')
# df['wind_direction'] = df['wind_direction2'].copy()
df = df.drop('wind_direction', axis=1)

# **12. Extract representative weather features**
### Features include clear, fair, cloudy, windy, rain, snow, obscuration, and sand storm.

In [5]:
t = df['weather_condition'].value_counts().compute()

In [6]:
t[:50]

Fair                            899305
Clear                           808180
Mostly Cloudy                   571419
Partly Cloudy                   397204
Overcast                        382485
Cloudy                          322809
Light Rain                      206281
Scattered Clouds                204661
Light Snow                       57107
Rain                             48614
Haze                             46756
Fog                              45833
Heavy Rain                       18215
Light Drizzle                    14582
Fair / Windy                     11380
Smoke                             8321
Mostly Cloudy / Windy             6529
Snow                              6425
Cloudy / Windy                    6262
T-Storm                           5015
Light Thunderstorms and Rain      4927
Thunderstorm                      4440
Thunder in the Vicinity           4329
Light Rain with Thunder           3980
Partly Cloudy / Windy             3871
Thunder                  

In [9]:
df['weather_condition'] = df['weather_condition'].fillna('na').str.lower()
df['weather_clear'] = False
df['weather_clear'] = df['weather_clear'].mask(df['weather_condition'] == 'clear', True)

df['weather_fair'] = False
df['weather_fair'] = df['weather_fair'].mask((df['weather_condition'] == 'fair')|
                                             (df['weather_condition'] == 'scattered clouds')|                                             
                                             (df['weather_condition'] == 'n/a precipitation'), True)

df['weather_cloudy'] = False
df['weather_cloudy'] = df['weather_cloudy'].mask((df['weather_condition'].str.contains('cloudy'))|
                                                 (df['weather_condition'].str.contains('overcast')), 'True')

df['weather_windy'] = False
df['weather_windy'] = df['weather_windy'].mask((df['weather_condition'].str.contains('windy'))|
                                               (df['weather_condition'].str.contains('squalls'))|
                                               (df['weather_condition'].str.contains('tornado')), 'True')

df['weather_rain'] = False
df['weather_rain'] = df['weather_rain'].mask((df['weather_condition'].str.contains('rain'))|
                                             (df['weather_condition'].str.contains('drizzle'))|
                                             (df['weather_condition'].str.contains('t-storm'))|
                                             (df['weather_condition'].str.contains('thunderstorms and rain'))|
                                             (df['weather_condition'].str.contains('hail'))|
                                             (df['weather_condition'].str.contains('in the vicinity'))|
                                             (df['weather_condition'] == 'thunder')|
                                             (df['weather_condition'].str.contains('light thunderstorm')), 'True')
df['weather_snow'] = False
df['weather_snow'] = df['weather_snow'].mask((df['weather_condition'].str.contains('snow'))|
                                             (df['weather_condition'].str.contains('wintry'))|
                                             (df['weather_condition'].str.contains('sleet'))|
                                             (df['weather_condition'].str.contains('ice')), 'True')

df['weather_obscuration'] = False
df['weather_obscuration'] = df['weather_obscuration'].mask((df['weather_condition'].str.contains('fog'))|
                                                           (df['weather_condition'].str.contains('haze'))|
                                                           (df['weather_condition'].str.contains('smoke'))|
                                                           (df['weather_condition'].str.contains('dust'))|
                                                           (df['weather_condition'].str.contains('mist')), 'True')

df['weather_sand'] = False
df['weather_sand'] = df['weather_sand'].mask((df['weather_condition'].str.contains('sand'))|
                                             (df['weather_condition'].str.contains('dust')), 'True')

df['weather_na'] = False
df['weather_na'] = df['weather_na'].mask((df['weather_condition'] == 'na'), True)
df = df.drop('weather_condition', axis=1)

In [12]:
df = df.compute()
df.rename(columns={'wind_direction2': 'wind_direction'}, inplace=True)

# **13. Save a csv file with clean data**

In [13]:
fn = '../data/clean_data/us_accidents.csv'
df.to_csv(fn, index=False)

In [14]:
df.shape

(4229393, 46)

In [15]:
df.head(3)

,severity,start_lat,start_lng,distance(mi),side,state,temperature(f),humidity(%),pressure(in),visibility(mi),...,wind_direction,weather_clear,weather_fair,weather_cloudy,weather_windy,weather_rain,weather_snow,weather_obscuration,weather_sand,weather_na
0,3,39.88,-84.06,0.01,R,OH,36.9,91.0,29.69,10.0,...,calm,False,False,False,False,True,False,False,False,False
1,2,39.94,-82.80,0.01,L,OH,37.9,100.0,29.66,10.0,...,calm,False,False,False,False,True,False,False,False,False
2,2,39.06,-84.06,0.01,R,OH,36.0,100.0,29.67,10.0,...,sw,False,False,True,False,False,False,False,False,False
